In [6]:
#!pip install pybind11


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
#!git clone https://github.com/varnerlab/dfba.git

Cloning into 'dfba'...
remote: Repository not found.
fatal: repository 'https://github.com/varnerlab/dfba.git/' not found


In [11]:
#pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.7 MB/s eta 0:00:00eta 0:00:01

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from tqdm import tqdm

from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import cobra
from cobra.io import load_model
model = load_model('iND750.xml')

RuntimeError: The model 'iND750.xml' could not be found in any of the repositories.

In [12]:
%run Scaffold_Y_lipo.ipynb


In [28]:
def add_dynamic_bounds(model, y):
    biomass, EPA = y  # expand the boundary species
    Glc_max_import = -10 * EPA / (5 + EPA)
    model.reactions.DM_Eicosapentaenoic_Acid.lower.bound = EPA_max_import


def dynamic_system(t, y):
    """Calculate the time derivative of external species."""

    biomass, EPA = y  # expand the boundary species

    # Calculate the specific exchanges fluxes at the given external concentrations.
    with model:
        add_dynamic_bounds(model, y)

        cobra.util.add_lp_feasibility(model)
        feasibility = cobra.util.fix_objective_as_constraint(model)
        lex_constraints = cobra.util.add_lexicographic_constraints(
             model, ['BIOMASS_SC4_bal', 'EPA'], ['max', 'max'])
    fluxes = lex_constraints.values
    fluxes *= biomass
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description('t = {:.3f}'.format(t))

    return fluxes

In [29]:
dynamic_system.pbar = None

In [30]:
def infeasible_event(t, y):
    with model:

        add_dynamic_bounds(model, y)

        cobra.util.add_lp_feasibility(model)
        feasibility = cobra.util.fix_objective_as_constraint(model)

    return feasibility - infeasible_event.epsilon

infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

In [31]:
ts = np.linspace(0, 15, 100)  # Desired integration resolution and interval
y0 = [0.1, 10]

with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF'
    )


0it [00:00, ?it/s]


AttributeError: 'Reaction' object has no attribute 'lower'